### Protótipo de ETL no contexto dos dados estatísticos da covid-19 nos continentes

In [ ]:
import pandas as pd

df = pd.read_csv(r'../csv/covid_continents.csv', index_col=0)

dataframe = df.copy()

# pd.set_option('display.max_columns', None)   
# pd.set_option('display.max_rows', None)      
# pd.set_option('display.width', 300)         
# pd.set_option('display.max_colwidth', None) 
# pd.set_option('display.float_format', '{:.2f}'.format)

dataframe.columns


Index(['updated', 'cases', 'todayCases', 'deaths', 'todayDeaths', 'recovered',
       'todayRecovered', 'active', 'critical', 'casesPerOneMillion',
       'deathsPerOneMillion', 'tests', 'testsPerOneMillion', 'population',
       'continent', 'activePerOneMillion', 'recoveredPerOneMillion',
       'criticalPerOneMillion', 'continentInfo', 'countries'],
      dtype='object')

# 🧪 Atividade Prática – Tratamento e Padronização de Dados COVID-19 por Continente

## 🎯 Objetivo geral
Tratar, validar e padronizar cada coluna do dataset garantindo:
- Tipos corretos (`dtype`)
- Consistência semântica
- Ausência de valores inválidos
- Prontidão para análise

---

## 1️⃣ Tradução e compreensão das colunas (snake_case)

### Objetivo
Padronizar os nomes das colunas para `snake_case` e garantir entendimento semântico
antes do tratamento dos dados.

| Coluna original | Coluna tratada (snake_case) | Significado |
|-----------------|----------------------------|-------------|
| updated | ultima_atualizacao | Timestamp da última atualização (epoch em ms) |
| cases | casos_totais | Total de casos acumulados |
| todayCases | casos_hoje | Casos registrados no dia |
| deaths | obitos_totais | Total de óbitos acumulados |
| todayDeaths | obitos_hoje | Óbitos registrados no dia |
| recovered | recuperados | Total de recuperados |
| todayRecovered | recuperados_hoje | Recuperados no dia |
| active | casos_ativos | Casos ativos |
| critical | casos_criticos | Casos críticos |
| casesPerOneMillion | casos_por_milhao_habitantes | Casos por milhão de habitantes |
| deathsPerOneMillion | obitos_por_milhao_habitantes | Óbitos por milhão de habitantes |
| tests | total_testes | Total de testes realizados |
| testsPerOneMillion | testes_por_milhao_habitantes | Testes por milhão de habitantes |
| population | populacao_total | População total do continente |
| continent | continente | Nome do continente |
| activePerOneMillion | casos_ativos_por_milhao_habitantes | Casos ativos por milhão |
| recoveredPerOneMillion | recuperados_por_milhao_habitantes | Recuperados por milhão |
| criticalPerOneMillion | criticos_por_milhao_habitantes | Casos críticos por milhão |
| continentInfo | continente_info | Informações geográficas do continente (lat/long) |
| countries | paises | Lista de países pertencentes ao continente |

### Atividades
- [x] Renomear todas as colunas para `snake_case`
- [x] Confirmar que os nomes refletem corretamente o significado dos dados

---

## 2️⃣ Coluna: `ultima_atualizacao` (`int64`)

**Descrição:** timestamp em epoch (milissegundos).

- [x] Garantir tipo `int64`
- [x] Verificar se os valores possuem **13 dígitos**
- [x] Identificar valores nulos ou zero
- [x] Converter para `datetime` (`pd.to_datetime(unit="ms", utc=True)`)

---

## 3️⃣ Coluna: `casos_totais` (`int64`)

- [x] Garantir tipo `int32`
- [x] Tratar valores nulos
- [x] Substituir valores negativos por `0`
- [x] Validar: `casos_totais ≥ casos_ativos`

---

## 4️⃣ Coluna: `casos_hoje` (`int64`)

- [x] Garantir tipo `int32`
- [x] Tratar valores nulos
- [x] Substituir valores negativos por `0`
- [x] Validar: `casos_hoje ≤ casos_totais`

---

## 5️⃣ Coluna: `obitos_totais` (`int64`)

- [x] Garantir tipo `int32`
- [x] Tratar valores nulos
- [x] Substituir valores negativos por `0`
- [x] Validar: `obitos_totais ≤ casos_totais`

---

## 6️⃣ Coluna: `obitos_hoje` (`int64`)

- [x] Garantir tipo `int32`
- [x] Tratar valores nulos
- [x] Substituir valores negativos por `0`
- [x] Validar: `obitos_hoje ≤ obitos_totais`

---

## 7️⃣ Coluna: `recuperados` (`int64`)

- [x] Garantir tipo `int32`
- [x] Tratar valores nulos
- [x] Substituir valores negativos por `0`
- [x] Validar: `recuperados ≤ casos_totais`

---

## 8️⃣ Coluna: `recuperados_hoje` (`int64`)

- [x] Garantir tipo `int32`
- [x] Tratar valores nulos
- [x] Substituir valores negativos por `0`
- [x] Validar: `recuperados_hoje ≤ recuperados`

---

## 9️⃣ Coluna: `casos_ativos` (`int64`)

- [x] Garantir tipo `int32`
- [x] Tratar valores nulos
- [x] Substituir valores negativos por `0`

---

## 🔟 Coluna: `casos_criticos` (`int64`)

- [x] Garantir tipo `int32`
- [x] Tratar valores nulos
- [x] Substituir valores negativos por `0`
- [x] Validar: `casos_criticos ≤ casos_ativos`

---

## 1️⃣1️⃣ Coluna: `casos_por_milhao_habitantes` (`float64`)

- [x] Garantir tipo `float32`
- [x] Tratar valores nulos
- [x] Substituir valores negativos por `0`

---

## 1️⃣2️⃣ Coluna: `obitos_por_milhao_habitantes` (`float64`)

- [x] Garantir tipo `float32`
- [x] Tratar valores nulos
- [x] Substituir valores negativos por `0`

---

## 1️⃣3️⃣ Coluna: `total_testes` (`int64`)

- [x] Garantir tipo `int64`
- [x] Tratar valores nulos
- [x] Substituir valores negativos por `0`
- [x] Validar: `total_testes ≥ casos_totais`

---

## 1️⃣4️⃣ Coluna: `testes_por_milhao_habitantes` (`float64`)

- [x] Garantir tipo `float64`
- [x] Tratar valores nulos
- [x] Substituir valores negativos por `0`

---

## 1️⃣5️⃣ Coluna: `populacao_total` (`int64`)

- [x] Garantir tipo `int64`
- [x] Tratar valores nulos
- [x] Substituir valores ≤ 0 por `NaN`
- [x] Validar se a população é plausível (> 1.000.000)

---

## 1️⃣6️⃣ Coluna: `continente` (`object`)

- [x] Converter para `string`
- [x] Remover espaços extras
- [x] Padronizar capitalização
- [x] Validar valores permitidos:
- África
- América do Norte
- América do Sul
- Europa
- Ásia
- Oceania

---

## 1️⃣7️⃣ Coluna: `casos_ativos_por_milhao_habitantes` (`float64`)

- [x] Garantir tipo `float32`
- [x] Tratar valores nulos
- [x] Substituir valores negativos por `0`

---

## 1️⃣8️⃣ Coluna: `recuperados_por_milhao_habitantes` (`float64`)

- [x] Garantir tipo `float32`
- [x] Tratar valores nulos
- [x] Substituir valores negativos por `0`

---

## 1️⃣9️⃣ Coluna: `criticos_por_milhao_habitantes` (`float64`)

- [x] Garantir tipo `float32`
- [x] Tratar valores nulos
- [x] Substituir valores negativos por `0`

---

## 2️⃣0️⃣ Coluna: `continente_info` (`object`)

- [x] Converter string para dicionário (`ast.literal_eval`)
- [x] Extrair:
- `lat` → `continente_lat`
- `long` → `continente_long`
- [x] Validar intervalos geográficos
- [x] Remover coluna original `continente_info`

---

## 2️⃣1️⃣ Coluna: `paises` (`object`)

- [x] Tratar a coluna com valores vazios
- [x] Garantir que seja letra maiuscula

---


# Tradução das colunas e transformando em snake_case

In [2]:
dataframe.columns

column_translate = {
    'updated': 'ultima_atualizacao',
    'cases': 'casos_totais',
    'todayCases': 'casos_hoje',
    'deaths': 'obitos_totais',
    'todayDeaths': 'obitos_hoje', 
    'recovered': 'recuperados',
    'todayRecovered': 'recuperados_hoje', 
    'active': 'casos_ativos', 
    'critical': 'casos_criticos', 
    'casesPerOneMillion': 'casos_por_milhao_habitantes',
    'deathsPerOneMillion': 'obitos_por_milhao_habitantes', 
    'tests': 'total_testes', 
    'testsPerOneMillion': 'testes_por_milhao_habitantes',
    'population': 'populacao_total', 
    'continent': 'continente', 
    'activePerOneMillion': 'casos_ativos_por_milhao_habitantes',
    'recoveredPerOneMillion': 'recuperados_por_milhao_habitantes',
    'criticalPerOneMillion': 'criticos_por_milhao_habitantes',
    'continentInfo': 'continente_info',
    'countries': 'paises'
}


dataframe = dataframe.rename(columns=column_translate)
dataframe.columns

Index(['ultima_atualizacao', 'casos_totais', 'casos_hoje', 'obitos_totais',
       'obitos_hoje', 'recuperados', 'recuperados_hoje', 'casos_ativos',
       'casos_criticos', 'casos_por_milhao_habitantes',
       'obitos_por_milhao_habitantes', 'total_testes',
       'testes_por_milhao_habitantes', 'populacao_total', 'continente',
       'casos_ativos_por_milhao_habitantes',
       'recuperados_por_milhao_habitantes', 'criticos_por_milhao_habitantes',
       'continente_info', 'paises'],
      dtype='object')

# Exploração inicial do dataset

In [3]:
dataframe.shape

#(6, 20)

dataframe.dtypes

# ultima_atualizacao                      int64
# casos_totais                            int64
# casos_hoje                              int64
# obitos_totais                           int64
# obitos_hoje                             int64
# recuperados                             int64
# recuperados_hoje                        int64
# casos_ativos                            int64
# casos_criticos                          int64
# casos_por_milhao_habitantes           float64
# obitos_por_milhao_habitantes          float64
# total_testes                            int64
# testes_por_milhao_habitantes          float64
# populacao_total                         int64
# continente                             object
# casos_ativos_por_milhao_habitantes    float64
# recuperados_por_milhao_habitantes     float64
# criticos_por_milhao_habitantes        float64
# continente_info                        object
# paises                                 object
# dtype: object

dataframe.isna().sum()

# Não há registros com valores nulos

ultima_atualizacao                    0
casos_totais                          0
casos_hoje                            0
obitos_totais                         0
obitos_hoje                           0
recuperados                           0
recuperados_hoje                      0
casos_ativos                          0
casos_criticos                        0
casos_por_milhao_habitantes           0
obitos_por_milhao_habitantes          0
total_testes                          0
testes_por_milhao_habitantes          0
populacao_total                       0
continente                            0
casos_ativos_por_milhao_habitantes    0
recuperados_por_milhao_habitantes     0
criticos_por_milhao_habitantes        0
continente_info                       0
paises                                0
dtype: int64

# Coluna ultima_atualizacao

In [4]:
dataframe['ultima_atualizacao'] = dataframe['ultima_atualizacao'].astype('int64')

dataframe['ultima_atualizacao'].astype(str).str.len().unique()

# array([13]) todas contém 13 digitos

dataframe['ultima_atualizacao'].isna().sum()

# np.int64(0)

dataframe['ultima_atualizacao'] = pd.to_datetime(arg=dataframe['ultima_atualizacao'], unit='ms', utc=True)

dataframe['ultima_atualizacao']

0   2025-12-14 21:25:38.353000+00:00
1   2025-12-14 21:25:38.354000+00:00
2   2025-12-14 21:25:38.355000+00:00
3   2025-12-14 21:25:38.356000+00:00
4   2025-12-14 21:25:38.357000+00:00
5   2025-12-14 21:25:38.358000+00:00
Name: ultima_atualizacao, dtype: datetime64[ns, UTC]

# Coluna Casos_totais

In [5]:
dataframe['casos_totais'] = dataframe['casos_totais'].astype('int32')


dataframe['casos_totais'].isna().sum()

# np.int64(0)

(dataframe['casos_totais'] >= dataframe['casos_ativos']).value_counts()

# 6

dataframe['casos_totais'] = dataframe['casos_totais'].fillna(0)


(dataframe['casos_totais'] < 0).sum()

# 0

dataframe['casos_totais']

0    131889132
1    221500265
2    253406198
3     70200879
4     14895771
5     12860924
Name: casos_totais, dtype: int32

# Coluna casos_hoje

In [6]:
dataframe['casos_hoje'] = dataframe['casos_hoje'].astype('int32')


dataframe['casos_hoje'].isna().sum()

# np.int64(0)

(dataframe['casos_hoje'] <= dataframe['casos_totais']).value_counts()

# 6

dataframe['casos_hoje'] = dataframe['casos_hoje'].fillna(0)


(dataframe['casos_hoje'] < 0).sum()

# 0

dataframe['casos_hoje']

0    0
1    0
2    0
3    0
4    0
5    0
Name: casos_hoje, dtype: int32

# Coluna obitos_totais 

In [7]:
dataframe['obitos_totais'] = dataframe['obitos_totais'].astype('int32')


dataframe['obitos_totais'].isna().sum()

# np.int64(0)

(dataframe['obitos_totais'] <= dataframe['casos_totais']).value_counts()

# # 6

dataframe['obitos_totais'] = dataframe['obitos_totais'].fillna(0)


(dataframe['obitos_totais'] < 0).sum()

# 0

dataframe['obitos_totais']

0    1695941
1    1553662
2    2101824
3    1367332
4      33015
5     258892
Name: obitos_totais, dtype: int32

# Coluna obitos_hoje

In [8]:
dataframe['obitos_hoje'] = dataframe['obitos_hoje'].astype('int32')


dataframe['obitos_hoje'].isna().sum()

# np.int64(0)

(dataframe['obitos_hoje'] <= dataframe['casos_totais']).value_counts()

# 6

dataframe['obitos_hoje'] = dataframe['obitos_hoje'].fillna(0)


(dataframe['obitos_hoje'] < 0).sum()

# 0

dataframe['obitos_hoje']

0    0
1    0
2    0
3    0
4    0
5    0
Name: obitos_hoje, dtype: int32

# Coluna recuperados


In [9]:
dataframe['recuperados'] = dataframe['recuperados'].astype('int32')


dataframe['recuperados'].isna().sum()

# np.int64(0)

dataframe['recuperados'] = dataframe['recuperados'].fillna(0)


(dataframe['recuperados'] < 0).sum()

# 0

dataframe['recuperados']

0    127665129
1    205673091
2    248754104
3     66683585
4     14752388
5     12090808
Name: recuperados, dtype: int32

# Coluna recuperados_hoje

In [10]:
dataframe['recuperados_hoje'] = dataframe['recuperados_hoje'].astype('int32')


dataframe['recuperados_hoje'].isna().sum()

# np.int64(0)

dataframe['recuperados_hoje'] = dataframe['recuperados_hoje'].fillna(0)

(dataframe['recuperados_hoje'] <=  dataframe['recuperados']).sum()

# 6

(dataframe['recuperados_hoje'] < 0).sum()

# 0

dataframe['recuperados_hoje']

0    350
1      0
2    474
3      0
4      0
5      0
Name: recuperados_hoje, dtype: int32

# Coluna casos_ativos

In [11]:
dataframe['casos_ativos'] = dataframe['casos_ativos'].astype('int32')


dataframe['casos_ativos'].isna().sum()

# np.int64(0)

dataframe['casos_ativos'] = dataframe['casos_ativos'].fillna(0)

# 6

(dataframe['casos_ativos'] < 0).sum()

#  0

dataframe['casos_ativos']

0     2528062
1    14273512
2     2550270
3     2149962
4      110368
5      511224
Name: casos_ativos, dtype: int32

# Coluna casos_criticos

In [12]:

dataframe['casos_criticos'] = dataframe['casos_criticos'].astype('int32')

dataframe['casos_criticos'] = dataframe['casos_criticos'].fillna(0)

dataframe['casos_criticos'] = dataframe['casos_criticos'].clip(lower=0)

(dataframe['casos_criticos'] <= dataframe['casos_ativos']).sum()

# np.int64(6)

dataframe['casos_criticos']

0     6095
1    14733
2     4453
3     8953
4       31
5      529
Name: casos_criticos, dtype: int32

# Coluna casos_por_milhao_habitantes

In [13]:
dataframe['casos_por_milhao_habitantes'] = dataframe['casos_por_milhao_habitantes'].astype('float32')

dataframe['casos_por_milhao_habitantes'] = dataframe['casos_por_milhao_habitantes'].fillna(0.0)

dataframe['casos_por_milhao_habitantes'] = dataframe['casos_por_milhao_habitantes'].clip(lower=0)

dataframe['casos_por_milhao_habitantes']

0    220498.437500
1     47014.109375
2    338985.062500
3    160387.875000
4    342664.625000
5      9142.429688
Name: casos_por_milhao_habitantes, dtype: float32

# Coluna obitos_por_milhao_habitantes

In [14]:
dataframe['obitos_por_milhao_habitantes'] = dataframe['obitos_por_milhao_habitantes'].astype('float32')

dataframe['obitos_por_milhao_habitantes'] = dataframe['obitos_por_milhao_habitantes'].fillna(0.0)

dataframe['obitos_por_milhao_habitantes'] = dataframe['obitos_por_milhao_habitantes'].clip(lower=0)

dataframe['obitos_por_milhao_habitantes']


0    2835.350098
1     329.769989
2    2811.639893
3    3123.939941
4     759.479980
5     184.039993
Name: obitos_por_milhao_habitantes, dtype: float32

# Coluna total_testes

In [15]:
dataframe['total_testes'] = dataframe['total_testes'].astype('int64')

dataframe['total_testes'] = dataframe['total_testes'].fillna(0)

dataframe['total_testes'] = dataframe['total_testes'].clip(lower=0)

dataframe['total_testes']

0    1325710449
1    2404652039
2    2848046702
3     245395353
4      91668192
5     111032578
Name: total_testes, dtype: int64

# Coluna testes_por_milhao_habitantes

In [16]:
dataframe['testes_por_milhao_habitantes'] = dataframe['testes_por_milhao_habitantes'].astype('float32')

dataframe['testes_por_milhao_habitantes'] = dataframe['testes_por_milhao_habitantes'].fillna(0.0)

dataframe['testes_por_milhao_habitantes'] = dataframe['testes_por_milhao_habitantes'].clip(lower=0.0)

dataframe['testes_por_milhao_habitantes']

0    2.216385e+06
1    5.103948e+05
2    3.809872e+06
3    5.606545e+05
4    2.108749e+06
5    7.892963e+04
Name: testes_por_milhao_habitantes, dtype: float32

# Coluna populacao_total

In [17]:
dataframe['populacao_total'] = dataframe['populacao_total'].astype('int32')

dataframe['populacao_total'] = dataframe['populacao_total'].fillna(0)
dataframe['populacao_total'] = dataframe['populacao_total'].clip(lower=0.0)

len(dataframe[dataframe['populacao_total'] > 1_000_000])



6

# Coluna continente

In [18]:
rename_continente = {
    'North America': 'América do Norte',
    'Asia': 'Ásia',
    'Europe': 'Europa',
    'South America': 'América do Sul',
    'Australia-Oceania': 'Oceania',
    'Africa': 'África'
}

dataframe['continente'] =  dataframe['continente'].replace(rename_continente)

dataframe['continente'] = (dataframe['continente'].astype('str').str.strip().str.title())

dataframe['continente']

0    América Do Norte
1                Ásia
2              Europa
3      América Do Sul
4             Oceania
5              África
Name: continente, dtype: object

# Coluna casos_ativos_por_milhao_habitantes

In [19]:
dataframe['casos_ativos_por_milhao_habitantes'] = dataframe['casos_ativos_por_milhao_habitantes'].astype('float32')

dataframe['casos_ativos_por_milhao_habitantes'] = dataframe['casos_ativos_por_milhao_habitantes'].fillna(0.0)

dataframe['casos_ativos_por_milhao_habitantes'] = dataframe['casos_ativos_por_milhao_habitantes'].clip(lower=0)

dataframe['casos_ativos_por_milhao_habitantes']

0    4226.529785
1    3029.600098
2    3411.530029
3    4912.020020
4    2538.919922
5     363.410004
Name: casos_ativos_por_milhao_habitantes, dtype: float32

# Coluna recuperados_por_mulhao_habitantes

In [20]:
dataframe['recuperados_por_milhao_habitantes'] = dataframe['recuperados_por_milhao_habitantes'].astype('float32')

dataframe['recuperados_por_milhao_habitantes'] = dataframe['recuperados_por_milhao_habitantes'].fillna(0.0)

dataframe['recuperados_por_milhao_habitantes'] = dataframe['recuperados_por_milhao_habitantes'].clip(lower=0)

dataframe['recuperados_por_milhao_habitantes']

0    213436.546875
1     43654.750000
2    332761.906250
3    152351.906250
4    339366.218750
5      8594.980469
Name: recuperados_por_milhao_habitantes, dtype: float32

# Coluna criticos_por_milhao_habitantes

In [21]:
dataframe['criticos_por_milhao_habitantes'] = dataframe['criticos_por_milhao_habitantes'].astype('float32')

dataframe['criticos_por_milhao_habitantes'] = dataframe['criticos_por_milhao_habitantes'].fillna(0.0)

dataframe['criticos_por_milhao_habitantes'] = dataframe['criticos_por_milhao_habitantes'].clip(lower=0)

dataframe['criticos_por_milhao_habitantes']

0    10.190000
1     3.130000
2     5.960000
3    20.450001
4     0.710000
5     0.380000
Name: criticos_por_milhao_habitantes, dtype: float32

# Coluna continente_info

In [22]:
import ast

dataframe['continente_info'] =  dataframe['continente_info'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

dataframe['continente_lat'] = dataframe['continente_info'].apply(
    lambda x: x.get('lat')
)

dataframe['continente_long'] = dataframe['continente_info'].apply(
    lambda x: x.get('long')
)

del dataframe['continente_info']

dataframe

,ultima_atualizacao,casos_totais,casos_hoje,obitos_totais,obitos_hoje,recuperados,recuperados_hoje,casos_ativos,casos_criticos,casos_por_milhao_habitantes,...,total_testes,testes_por_milhao_habitantes,populacao_total,continente,casos_ativos_por_milhao_habitantes,recuperados_por_milhao_habitantes,criticos_por_milhao_habitantes,paises,continente_lat,continente_long
0,2025-12-14 21:25:38.353000+00:00,131889132,0,1695941,0,127665129,350,2528062,6095,220498.437500,...,1325710449,2.216385e+06,598140916,América Do Norte,4226.529785,213436.546875,10.190000,"['Anguilla', 'Antigua and Barbuda', 'Aruba', '...",31.676827,-146.470747
1,2025-12-14 21:25:38.354000+00:00,221500265,0,1553662,0,205673091,0,14273512,14733,47014.109375,...,2404652039,5.103948e+05,416389487,Ásia,3029.600098,43654.750000,3.130000,"['Afghanistan', 'Armenia', 'Azerbaijan', 'Bahr...",23.702727,62.375064
2,2025-12-14 21:25:38.355000+00:00,253406198,0,2101824,0,248754104,474,2550270,4453,338985.062500,...,2848046702,3.809872e+06,747543837,Europa,3411.530029,332761.906250,5.960000,"['Albania', 'Andorra', 'Austria', 'Belarus', '...",25.771324,-35.601226
3,2025-12-14 21:25:38.356000+00:00,70200879,0,1367332,0,66683585,0,2149962,8953,160387.875000,...,245395353,5.606545e+05,437694443,América Do Sul,4912.020020,152351.906250,20.450001,"['Argentina', 'Bolivia', 'Brazil', 'Chile', 'C...",-15.655156,-100.748423
4,2025-12-14 21:25:38.357000+00:00,14895771,0,33015,0,14752388,0,110368,31,342664.625000,...,91668192,2.108749e+06,43470408,Oceania,2538.919922,339366.218750,0.710000,"['Australia', 'Cook Islands', 'Fiji', 'French ...",-8.659916,91.146985
5,2025-12-14 21:25:38.358000+00:00,12860924,0,258892,0,12090808,0,511224,529,9142.429688,...,111032578,7.892963e+04,1406728744,África,363.410004,8594.980469,0.380000,"['Algeria', 'Angola', 'Benin', 'Botswana', 'Bu...",1.738387,-16.309464


# Coluna países

In [23]:
def clean_lista_paises(x):
    if not isinstance(x, list):
        return x

    return [
        'Não informado' if pais.strip() == ''
        else pais.strip().title()
        for pais in x
        if isinstance(pais, str)
    ]


dataframe['paises'] = dataframe['paises'].apply(
    clean_lista_paises
)

dataframe['paises']

0    ['Anguilla', 'Antigua and Barbuda', 'Aruba', '...
1    ['Afghanistan', 'Armenia', 'Azerbaijan', 'Bahr...
2    ['Albania', 'Andorra', 'Austria', 'Belarus', '...
3    ['Argentina', 'Bolivia', 'Brazil', 'Chile', 'C...
4    ['Australia', 'Cook Islands', 'Fiji', 'French ...
5    ['Algeria', 'Angola', 'Benin', 'Botswana', 'Bu...
Name: paises, dtype: object